In [1]:
import sys
import os
import shutil
from collections import namedtuple
sys.path.append('..') # assuming we're running from OpenNIR/examples/
os.environ['ONIR_IGNORE_ARGV'] = 'true' # don't process command line arguments (they come from jupyter)

In [2]:
import pandas as pd
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir.pt

PyTerrier 0.3.1 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
# Sample EPIC re-ranker (trained on msmarco)
rr = onir.pt.reranker.from_checkpoint('epic.msmarco.tar.gz', {'learning_rate': 1e-5})

configuraiton file not found: config


In [4]:
dataset = pt.datasets.get_dataset('irds:vaswani')
index_path = './index_vaswani'
if not os.path.exists(index_path):
    indexer = pt.index.IterDictIndexer(index_path)
    index_ref = indexer.index(dataset.get_corpus_iter(), meta=('docno', 'text'))
else:
    index_ref = pt.IndexRef.of(index_path + '/data.properties')
index = pt.IndexFactory.of(index_ref)

In [5]:
BM25_br = pt.BatchRetrieve(index, wmodel="BM25") % 100
res = BM25_br.transform(dataset.get_topics())
res_with_text = pt.text.get_text(index, "text").transform(res)

In [6]:
pt.Utils.evaluate(res, dataset.get_qrels(), metrics = ['map'])

{'map': 0.2725231249761632}

In [7]:
epic_res = rr.transform(res_with_text)

[2021-02-21 19:23:21,509][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:23:21,607][onir.pt][DEBUG] [starting] batches


[2021-02-21 19:24:37,860][onir.pt][DEBUG] [finished] batches: [01:16] [2325it] [30.49it/s]


In [8]:
pt.Utils.evaluate(epic_res, dataset.get_qrels(), metrics = ['map'])

{'map': 0.22642311752488706}

In [9]:
# (over)fit 1 training iteration on the vaswani dataset
rr.fit(tr_topics=dataset.get_topics(), tr_qrels=dataset.get_qrels(), tr_run=res_with_text)

[2021-02-21 19:24:37,924][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:24:37,929][onir.pt][DEBUG] [starting] training
[2021-02-21 19:24:37,931][onir.pt][DEBUG] [starting] train pairs


[2021-02-21 19:25:42,574][onir.pt][DEBUG] [finished] train pairs: [01:05] [1024it] [15.84it/s]
[2021-02-21 19:25:42,578][onir.pt][DEBUG] [finished] training [01:05]
[2021-02-21 19:25:42,580][onir.pt][INFO] training   it=0 loss=0.1159


,it,loss
0,0,0.115943


In [ ]:
# (over)fit many training iterations on the vaswani dataset
rr.fit(tr_topics=dataset.get_topics(), tr_qrels=dataset.get_qrels(), tr_run=res_with_text, va_run=res_with_text, va_qrels=dataset.get_qrels())

[2021-02-21 19:25:42,612][onir.pt][DEBUG] [starting] validation
[2021-02-21 19:25:42,614][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:25:42,619][onir.pt][DEBUG] [starting] batches


[2021-02-21 19:26:59,902][onir.pt][DEBUG] [finished] batches: [01:17] [2325it] [30.08it/s]
[2021-02-21 19:26:59,925][onir.pt][DEBUG] [finished] validation [01:17]
[2021-02-21 19:26:59,927][onir.pt][INFO] pre-validation: 0.3242
[2021-02-21 19:27:00,334][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:27:00,343][onir.pt][DEBUG] [starting] training
[2021-02-21 19:27:00,345][onir.pt][DEBUG] [starting] train pairs


[2021-02-21 19:28:05,983][onir.pt][DEBUG] [finished] train pairs: [01:06] [1024it] [15.60it/s]
[2021-02-21 19:28:05,988][onir.pt][DEBUG] [finished] training [01:06]
[2021-02-21 19:28:05,991][onir.pt][INFO] training   it=0 loss=0.0634
[2021-02-21 19:28:05,993][onir.pt][DEBUG] [starting] validation
[2021-02-21 19:28:05,995][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:28:06,002][onir.pt][DEBUG] [starting] batches


[2021-02-21 19:29:23,237][onir.pt][DEBUG] [finished] batches: [01:17] [2325it] [30.10it/s]
[2021-02-21 19:29:23,257][onir.pt][DEBUG] [finished] validation [01:17]
[2021-02-21 19:29:24,009][onir.pt][INFO] validation it=0 map=0.3683 ndcg=0.5795 P_10=0.4742 <--
[2021-02-21 19:29:24,012][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:29:24,020][onir.pt][DEBUG] [starting] training
[2021-02-21 19:29:24,021][onir.pt][DEBUG] [starting] train pairs


[2021-02-21 19:30:29,460][onir.pt][DEBUG] [finished] train pairs: [01:05] [1024it] [15.65it/s]
[2021-02-21 19:30:29,463][onir.pt][DEBUG] [finished] training [01:05]
[2021-02-21 19:30:29,466][onir.pt][INFO] training   it=0 loss=0.0710
[2021-02-21 19:30:29,468][onir.pt][DEBUG] [starting] validation
[2021-02-21 19:30:29,469][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:30:29,476][onir.pt][DEBUG] [starting] batches


...

[2021-02-21 19:49:37,229][onir.pt][DEBUG] [finished] train pairs: [01:06] [1024it] [15.57it/s]
[2021-02-21 19:49:37,234][onir.pt][DEBUG] [finished] training [01:06]
[2021-02-21 19:49:37,240][onir.pt][INFO] training   it=0 loss=0.0242
[2021-02-21 19:49:37,243][onir.pt][DEBUG] [starting] validation
[2021-02-21 19:49:37,245][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:49:37,255][onir.pt][DEBUG] [starting] batches


[2021-02-21 19:50:54,375][onir.pt][DEBUG] [finished] batches: [01:17] [2325it] [30.15it/s]
[2021-02-21 19:50:54,399][onir.pt][DEBUG] [finished] validation [01:17]
[2021-02-21 19:50:55,176][onir.pt][INFO] validation it=0 map=0.5368 ndcg=0.6783 P_10=0.6774 <--
[2021-02-21 19:50:55,179][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:50:55,187][onir.pt][DEBUG] [starting] training
[2021-02-21 19:50:55,189][onir.pt][DEBUG] [starting] train pairs


[2021-02-21 19:52:00,214][onir.pt][DEBUG] [finished] train pairs: [01:05] [1024it] [15.75it/s]
[2021-02-21 19:52:00,219][onir.pt][DEBUG] [finished] training [01:05]
[2021-02-21 19:52:00,222][onir.pt][INFO] training   it=0 loss=0.0238
[2021-02-21 19:52:00,224][onir.pt][DEBUG] [starting] validation
[2021-02-21 19:52:00,226][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:52:00,233][onir.pt][DEBUG] [starting] batches


[2021-02-21 19:53:17,438][onir.pt][DEBUG] [finished] batches: [01:17] [2325it] [30.12it/s]
[2021-02-21 19:53:17,457][onir.pt][DEBUG] [finished] validation [01:17]
[2021-02-21 19:53:17,460][onir.pt][INFO] validation it=0 map=0.5320 ndcg=0.6768 P_10=0.6613
[2021-02-21 19:53:17,461][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:53:17,467][onir.pt][DEBUG] [starting] training
[2021-02-21 19:53:17,468][onir.pt][DEBUG] [starting] train pairs


[2021-02-21 19:54:23,356][onir.pt][DEBUG] [finished] train pairs: [01:06] [1024it] [15.54it/s]
[2021-02-21 19:54:23,361][onir.pt][DEBUG] [finished] training [01:06]
[2021-02-21 19:54:23,364][onir.pt][INFO] training   it=0 loss=0.0235
[2021-02-21 19:54:23,366][onir.pt][DEBUG] [starting] validation
[2021-02-21 19:54:23,367][onir.pt][DEBUG] using GPU (deterministic)
[2021-02-21 19:54:23,375][onir.pt][DEBUG] [starting] batches


In [ ]:
# Train on a pair iterator (also works with dataframe of same columns)
def tr_pairs():
    import ir_datasets
    ds = ir_datasets.load('msmarco-passage/train')
    queries = {q.query_id: q for q in ds.queries_iter()}
    docstore = ds.docs_store()
    for scoreddoc in ds.docpairs_iter():
        yield onir.pt.TrainPair(
            scoreddoc.query_id,
            queries[scoreddoc.query_id].text,
            scoreddoc.doc_id_a,
            docstore.get(scoreddoc.doc_id_a).text,
            scoreddoc.doc_id_b,
            docstore.get(scoreddoc.doc_id_b).text)
rr.fit(tr_pairs=tr_pairs())